<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
    h1 {
        line-height: 200%;
        font-family: 'Comic Sans MS', cursive, sans-serif;
        color: #ffb6c1; /* رنگ نارنجی پاستیلی */
    }
</style>
</head>
<body>
    <h1>Zahra Alinaghipour</h1>
</body>
</html>

In this project, we will build a natural language processing model to categorize products using textual data related to a product, such as its title.

| <b>Column Name</b> | <b>Description</b> |
| :---: | :---: |
| <code>name1</code> | Main product title (usually in Persian) |
| <code>name2</code> | Secondary/optional product title (usually in English) |
| <code>cat_id</code> | Main product category ID |

| <b>Category ID</b> | <b>Category Title</b> |
| :---: | :---: |
| <code>0</code> | Men's jackets, raincoats, and overcoats |
| <code>1</code> | Men's sweatshirts and hoodies |
| <code>2</code> | Analog and digital wristwatches |
| <code>3</code> | Wall, desk, and decorative clocks |
| <code>4</code> | Regular and smart watch accessories |
| <code>5</code> | Children's sweatshirts and hoodies |
| <code>6</code> | Children's jackets and overcoats |
| <code>7</code> | Men's sports sweatshirts |
| <code>8</code> | Men's sports sweatshirts and pants |
| <code>9</code> | Bags and shopping carts |
| <code>10</code> | Suitcases and bags |

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading/Loading the dataset files
train = pd.read_csv('/content/drive/MyDrive/Project 3/title_categorizer/data/torob_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Project 3/title_categorizer/data/torob_test.csv')

In [3]:
train.head()

,name1,name2,cat_id
0,کاپشن کوهنوردی پر سنگین نورث فیس,NaN,0
1,پالتو خزدار مردانه مارک اصلی,NaN,0
2,کاپشن مردانه مدل Bako,NaN,0
3,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,NaN,0
4,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,NaN,0


In [4]:
test.head()

,name1,name2
0,بگ پارچه ای دو رنگ چاپ,NaN
1,بادگیر کلاهدار ورزشی فرانسه France windrunner,NaN
2,ساعت دیواری هندسی,NaN
3,ساک ورزشی Nike مدل N20627,NaN
4,ساعت فلزی,NaN


In [5]:
print("Train set size:", len(train))
print("Test set size:", len(test))
print("Fraction:", len(train)/len(test))

Train set size: 8822
Test set size: 1201
Fraction: 7.345545378850957


In [6]:
!pip install DadmaTools

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.0.1 which is incompatible.


In [7]:
!pip install --upgrade torch torchvision


  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dadmatools 2.0.6 requires torch<=2.0.1, but you have torch 2.3.1 which is incompatible.


In [9]:
import dadmatools.pipeline.language as language

pips = 'tok, lem'
nlp = language.Pipeline(pips)

Loading pretrained XLM-Roberta, this may take a while...
Model fa_tokenizer exists in cache/dadmatools/fa_tokenizer.pt
Loading tokenizer for persian
Loading tagger for persian
Loading multi-word expander for persian
Loading lemmatizer for persian
Active language: persian


In [10]:
from dadmatools.normalizer import Normalizer

normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="",
    replace_number_with="",
    replace_url_with="",
    replace_mobile_number_with="",
    replace_emoji_with="",
    replace_home_number_with=""
)

In [11]:
train['normalized_text'] = train['name1'].apply(normalizer.normalize)
test['normalized_text'] = test['name1'].apply(normalizer.normalize)

In [12]:
train.head()

,name1,name2,cat_id,normalized_text
0,کاپشن کوهنوردی پر سنگین نورث فیس,NaN,0,کاپشن کوهنوردی پر سنگین نورث فیس
1,پالتو خزدار مردانه مارک اصلی,NaN,0,پالتو خزدار مردانه مارک اصلی
2,کاپشن مردانه مدل Bako,NaN,0,کاپشن مردانه مدل Bako
3,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,NaN,0,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...
4,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,NaN,0,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز


In [13]:
import nltk

from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
train['tokens'] = train['normalized_text'].apply(lambda x: word_tokenize(x))
test['tokens'] = test['normalized_text'].apply(lambda x: word_tokenize(x))

In [15]:
import re
train['tokens'] = train['tokens'].apply(lambda x: [re.sub(r'\d+', '', token) for token in x])
test['tokens'] = test['tokens'].apply(lambda x: [re.sub(r'\d+', '', token) for token in x])

In [16]:
train['tokens'] = train['tokens'].apply(lambda x: [token for token in x if len(token) >= 3])
test['tokens'] = test['tokens'].apply(lambda x: [token for token in x if len(token) >= 3])

In [17]:
train['tokens'] = train['tokens'].apply(lambda x: [token.lower() for token in x])
test['tokens'] = test['tokens'].apply(lambda x: [token.lower() for token in x])

In [18]:
train.head()

,name1,name2,cat_id,normalized_text,tokens
0,کاپشن کوهنوردی پر سنگین نورث فیس,NaN,0,کاپشن کوهنوردی پر سنگین نورث فیس,"[کاپشن, کوهنوردی, سنگین, نورث, فیس]"
1,پالتو خزدار مردانه مارک اصلی,NaN,0,پالتو خزدار مردانه مارک اصلی,"[پالتو, خزدار, مردانه, مارک, اصلی]"
2,کاپشن مردانه مدل Bako,NaN,0,کاپشن مردانه مدل Bako,"[کاپشن, مردانه, مدل, bako]"
3,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,NaN,0,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,"[کاپشن, سالامون, salomon, پارچه, گورتکس, کره, ..."
4,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,NaN,0,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,"[کاپشن‌‌‌, نظامی‌‌‌, آمریکایی‌‌‌, سبز]"


In [19]:
train['normalized_text'] = train['tokens'].apply(lambda x: ' '.join(x))
test['normalized_text'] = test['tokens'].apply(lambda x: ' '.join(x))

In [20]:
train.head()

,name1,name2,cat_id,normalized_text,tokens
0,کاپشن کوهنوردی پر سنگین نورث فیس,NaN,0,کاپشن کوهنوردی سنگین نورث فیس,"[کاپشن, کوهنوردی, سنگین, نورث, فیس]"
1,پالتو خزدار مردانه مارک اصلی,NaN,0,پالتو خزدار مردانه مارک اصلی,"[پالتو, خزدار, مردانه, مارک, اصلی]"
2,کاپشن مردانه مدل Bako,NaN,0,کاپشن مردانه مدل bako,"[کاپشن, مردانه, مدل, bako]"
3,کاپشن سالامون salomon پارچه گورتکس کره ای اعلا...,NaN,0,کاپشن سالامون salomon پارچه گورتکس کره اعلا با...,"[کاپشن, سالامون, salomon, پارچه, گورتکس, کره, ..."
4,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,NaN,0,کاپشن‌‌‌ نظامی‌‌‌ آمریکایی‌‌‌ سبز,"[کاپشن‌‌‌, نظامی‌‌‌, آمریکایی‌‌‌, سبز]"


In [21]:
train.isnull().sum()

name1                 0
name2              8226
cat_id                0
normalized_text       0
tokens                0
dtype: int64

In [22]:
test.isnull().sum()

name1                 0
name2              1113
normalized_text       0
tokens                0
dtype: int64

In [23]:
train['normalized_text'] = train['normalized_text'].fillna(train['name1'])
test['normalized_text'] = test['normalized_text'].fillna(test['name1'])

In [24]:
train.isnull().sum()

name1                 0
name2              8226
cat_id                0
normalized_text       0
tokens                0
dtype: int64

name1                 0
name2              8226
cat_id                0
normalized_text       0
tokens                0
dtype: int64

In [26]:
test.isnull().sum()

name1                 0
name2              1113
normalized_text       0
tokens                0
dtype: int64

In [27]:
train.to_csv('train_preprocessed.csv', index=False)
test.to_csv('test_preprocessed.csv', index=False)

In [28]:
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [54]:
print(train['normalized_text'].isnull().sum())
print(test['normalized_text'].isnull().sum())

train['normalized_text'] = train['normalized_text'].fillna('')
test['normalized_text'] = test['normalized_text'].fillna('')

0
0


In [55]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train['normalized_text'], train['cat_id'], test_size=0.12, random_state=42)
X_test = test['normalized_text'].to_numpy()


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

train_acc = clf.score(X_train_tfidf, y_train)
val_acc = clf.score(X_val_tfidf, y_val)
print(f'train acc: {train_acc}, validation acc: {val_acc}')

train acc: 0.9603246167718665, validation acc: 0.9263456090651558


In [34]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42).fit(X_train_tfidf, y_train)

train_acc = clf.score(X_train_tfidf, y_train)
val_acc = clf.score(X_val_tfidf, y_val)
print(f'train acc: {train_acc}, validation acc: {val_acc}')

train acc: 0.9765554553651938, validation acc: 0.959395656279509


In [35]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=42).fit(X_train_tfidf, y_train)

train_acc = clf.score(X_train_tfidf, y_train)
val_acc = clf.score(X_val_tfidf, y_val)
print(f'train acc: {train_acc}, validation acc: {val_acc}')

train acc: 0.9929151101378333, validation acc: 0.9612842304060434


In [36]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 42.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [36]:
import keras

vectorizer = keras.layers.TextVectorization(output_mode='tf_idf', ngrams=2, max_tokens=4096, pad_to_max_tokens=True)
vectorizer.adapt(X_train)

In [37]:
vocab = vectorizer.get_vocabulary()
vocab_size = vectorizer.vocabulary_size()
print(vocab_size)

4096


In [38]:
model = keras.Sequential([
    vectorizer,
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1028, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(11, activation='softmax')
])

In [39]:
model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.SGD(learning_rate=0.01),
    metrics = ['accuracy']
)

In [40]:
y_train_cat = keras.utils.to_categorical(y_train, 11)
y_val_cat = keras.utils.to_categorical(y_val, 11)

In [41]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train_cat,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping_cb]
)

Epoch 1/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.1026 - loss: 2.4128 - val_accuracy: 0.2946 - val_loss: 2.3086
Epoch 2/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2063 - loss: 2.3259 - val_accuracy: 0.4618 - val_loss: 2.2060
Epoch 3/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2795 - loss: 2.2293 - val_accuracy: 0.5845 - val_loss: 2.0378
Epoch 4/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3823 - loss: 2.0734 - val_accuracy: 0.7025 - val_loss: 1.7741
Epoch 5/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5319 - loss: 1.8383 - val_accuracy: 0.7602 - val_loss: 1.3749
Epoch 6/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6443 - loss: 1.4850 - val_accuracy: 0.7866 - val_loss: 1.0675
Epoch 7/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7465 - loss: 1.1177 - val_accuracy: 0.8848 - val_loss: 0.6920
Epoch 8/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7850 - loss: 0.9313 - val_accu

In [42]:
train_acc = model.evaluate(X_train, y_train_cat)[1]
val_acc = model.evaluate(X_val, y_val_cat)[1]
print(f'train acc: {train_acc}, validation acc: {val_acc}')

243/243 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9541 - loss: 0.1604
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9345 - loss: 0.1808
train acc: 0.9551719427108765, validation acc: 0.9433428049087524


In [43]:
max_length = 0
for tokens in train['tokens']:
  if len(tokens) > max_length:
    max_length = len(tokens)
print(f'Maximum length of tokens: {max_length}')

Maximum length of tokens: 540


In [44]:
tokens_length = []
for tokens in train['tokens']:
  tokens_length.append(len(tokens))

mean_length = sum(tokens_length) / len(tokens_length)
print(f'Mean length of tokens: {mean_length}')

Mean length of tokens: 52.62264792564044


In [45]:
MAX_LEN = 300
vectorizer = keras.layers.TextVectorization(ngrams=2, output_sequence_length=MAX_LEN)
vectorizer.adapt(X_train)

In [46]:
VOCAB_SIZE = vectorizer.vocabulary_size()
EMBEDDING_DIM = 300

In [47]:
model = keras.Sequential()
model.add(vectorizer)
model.add(keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, name='embedding'))
model.add(keras.layers.LSTM(256, return_sequences=True, name='lstm', dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.GlobalMaxPool1D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(11, activation='softmax'))

In [48]:
model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(learning_rate=0.01),
    metrics = ['accuracy']
)

In [49]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train_cat,
    batch_size=64,
    epochs=50,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stopping_cb],
    shuffle=True
)

Epoch 1/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 72s 545ms/step - accuracy: 0.6059 - loss: 1.1251 - val_accuracy: 0.9481 - val_loss: 0.1483
Epoch 2/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 69s 563ms/step - accuracy: 0.9596 - loss: 0.1326 - val_accuracy: 0.9518 - val_loss: 0.1656
Epoch 3/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 69s 569ms/step - accuracy: 0.9781 - loss: 0.0763 - val_accuracy: 0.9509 - val_loss: 0.1862
Epoch 4/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 81s 558ms/step - accuracy: 0.9858 - loss: 0.0520 - val_accuracy: 0.9575 - val_loss: 0.2615
Epoch 5/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 70s 572ms/step - accuracy: 0.9849 - loss: 0.0523 - val_accuracy: 0.9547 - val_loss: 0.3152
Epoch 6/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 80s 553ms/step - accuracy: 0.9851 - loss: 0.0648 - val_accuracy: 0.9462 - val_loss: 0.2751
Epoch 7/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 84s 569ms/step - accuracy: 0.9838 - loss: 0.0693 - val_accuracy: 0.9500 - val_loss: 0.2938


In [50]:
model.evaluate(X_val, y_val_cat)

34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.9559 - loss: 0.2844


[0.2614651620388031, 0.9575070738792419]

In [51]:
submission = pd.DataFrame()
submission['cat_id'] = clf.predict(X_test_tfidf)
submission.head()

,cat_id
0,9
1,7
2,3
3,10
4,3


In [52]:
test_pred = model.predict(X_test)
submission = pd.DataFrame({'cat_id': np.argmax(test_pred, axis=1)})
submission.head()

38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 158ms/step


,cat_id
0,1
1,7
2,3
3,10
4,3
